# Figure 6

## Imports

In [ ]:
import dolfin
import matplotlib.pyplot as plt
import numpy
import scipy
import sympy
import sys

import dolfin_mech                    as dmech
import micro_poro_structure_generator as gen

## Defining geometry and material parameters

In [ ]:
seeds_filename = "Fig6-seeds.dat"
mesh_filebasename = "Fig6-mesh"

domain_y = 1
domain_x = domain_y * numpy.sqrt(3)/1.5/2
thickness = 0.092

gen.generate_seeds_semi_regular(
    DoI = 0.,
    row = 1,
    domain_y = domain_y,
    seeds_filename = seeds_filename)
gen.generate_mesh_2D_rectangle_w_voronoi_inclusions(
    mesh_filename = mesh_filebasename,
    seeds_filename = seeds_filename,
    h = thickness,
    lcar = thickness/5,
    domain_x = domain_x,
    domain_y = domain_y,
    shift_y = 0.,
    remove_seeds = False)

mesh = dolfin.Mesh()
dolfin.XDMFFile(mesh_filebasename+".xdmf").read(mesh)

coord = mesh.coordinates()
xmax = max(coord[:,0]); xmin = min(coord[:,0])
ymax = max(coord[:,1]); ymin = min(coord[:,1])
vertices = numpy.array([[xmin, ymin],
                        [xmax, ymin],
                        [xmax, ymax],
                        [xmin, ymax]])
bbox = [xmin, xmax, ymin, ymax]

V0 = (xmax - xmin)*(ymax - ymin)

dV = dolfin.Measure("dx", domain=mesh)
Vs0 = dolfin.assemble(dolfin.Constant(1.)*dV)

Phis0 = Vs0/V0

In [ ]:
Es = 1
nus = 0.499
mat_params = {"model":"CGNH", "parameters":{"E":Es, "nu":nus}}

## Defining kinematics

In [ ]:
alpha_x, alpha_y, alpha_xy, alpha_yx = sympy.symbols('alpha_x alpha_y alpha_xy alpha_yx')
epsilon_x, epsilon_y, epsilon_xy = sympy.symbols('epsilon_x epsilon_y epsilon_xy')
epsilon = sympy.symbols('epsilon')
C00, C01, C10, C11 = sympy.symbols('C00 C01 C10 C11')
C_CM = sympy.Matrix([[C00, C01],
                     [C10, C11]])
I_C_CM = sympy.trace(C_CM) 
II_C_CM = (sympy.trace(C_CM)**2 - sympy.trace(C_CM**2))/2 
III_C_CM = sympy.det(C_CM) 
J_CM = sympy.sqrt(III_C_CM)
E_CM = (C_CM - sympy.eye(2))/2 
p = sympy.symbols('p')

C_CM_bar = J_CM**(-2/3)*C_CM
I_C_CM_bar = sympy.trace(C_CM_bar)
II_C_CM_bar = (sympy.trace(C_CM_bar)**2 - sympy.trace(C_CM_bar**2))/2 

F = sympy.Matrix(
        [[alpha_x,    0  ],\
        [   0   , alpha_y]])
J = F.det()
C = F.T * F
E = (C - sympy.eye(2))/2

## Computing linearized model response

In [ ]:
homogenization_problem = dmech.HomogenizationProblem(
    dim=2,
    mesh=mesh,
    mat_params=mat_params["parameters"],
    vertices=vertices,
    vol=V0,
    bbox=bbox)
lmbda_tilde, mu_tilde = homogenization_problem.get_lambda_and_mu()
kappa_tilde = homogenization_problem.get_kappa()
# beta = mu_tilde/2
# alpha = lmbda_tilde/4

sigma_lin = lmbda_tilde * (2*epsilon) + 2 * mu_tilde * epsilon

## Computing macroscopic model response

In [ ]:
W_skel = (lmbda_tilde/4) * (J_CM**2 - 1 - 2 * sympy.ln(J_CM)) + (mu_tilde/2) * (I_C_CM - 2 - 2 * sympy.ln(J_CM))
Sigma_CM = 2*sympy.diff(W_skel, C_CM) - p * J_CM * C_CM.inv()

Sigma_macro = Sigma_CM.subs(list(zip(C_CM, C)))
sigma_macro = F * Sigma_macro * F.T / J
sigma_macro = sigma_macro.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1) # MG20231123: Fix control!

Sigma_ter_CM = 2*sympy.diff(W_skel, C_CM)
Sigma_ter_macro = Sigma_ter_CM.subs(list(zip(C_CM, C)))
sigma_ter_macro = F * Sigma_ter_macro * F.T / J # MG20231123: This is needed, right?!
sigma_ter_macro = sigma_ter_macro.subs(alpha_x, epsilon + 1).subs(alpha_y, epsilon + 1) # MG20231123: Fix control!

## Computing microscopic model response

* material parameters
* $\epsilon_{xx}, \epsilon_{yy}$ $\longrightarrow$ macroscopic strain
* If no macroscopic strain is applied, `Macroscopic_strain` = <span style="color:lightblue">None</span>
* sigma_bar is the applied stress, otherwise:
\begin{equation}
    \sigma = \begin{bmatrix}
                0 & 0\\
                0 & 0
             \end{bmatrix}
\end{equation}

<!-- * $\gamma$ is the surface tension coefficient, otherwise it is equal to zero -->

In [ ]:
def global_response(mesh, mat_params, eps_xx, eps_yy, pf, Macroscopic_strain, sigma_bar, gamma):
    dim = 2
    bcs = "pbc"
    step_params = {"dt_ini":1e-1, "dt_min":1e-3}

    res_folder = sys.argv[0][:-3]
    res_basename  = sys.argv[0][:-3]

    res_basename = res_folder+"/"+res_basename
    verbose=1

    ################################################################### Mesh ###

    coord = mesh.coordinates()
    xmax = max(coord[:,0]); xmin = min(coord[:,0])
    ymax = max(coord[:,1]); ymin = min(coord[:,1])
    vol = (xmax - xmin)*(ymax - ymin)
    dV = dolfin.Measure("dx",domain=mesh)

    vertices = numpy.array([[xmin, ymin],
                         [xmax, ymin],
                         [xmax, ymax],
                         [xmin, ymax]])

    tol = 1E-8  
    vv = vertices
    a1 = vv[1,:]-vv[0,:] # first vector generating periodicity
    a2 = vv[3,:]-vv[0,:] # second vector generating periodicity
    # check if UC vertices form indeed a parallelogram
    assert numpy.linalg.norm(vv[2, :]-vv[3, :] - a1) <= tol
    assert numpy.linalg.norm(vv[2, :]-vv[1, :] - a2) <= tol
    bbox = [xmin, xmax, ymin, ymax]

    ################################################## Subdomains & Measures ###

    
    xmin_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmin, tol=tol)
    xmax_sd = dolfin.CompiledSubDomain("near(x[0], x0, tol) && on_boundary", x0=xmax, tol=tol)
    ymin_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymin, tol=tol)
    ymax_sd = dolfin.CompiledSubDomain("near(x[1], x0, tol) && on_boundary", x0=ymax, tol=tol)


    xmin_id = 1
    xmax_id = 2
    ymin_id = 3
    ymax_id = 4

    boundaries_mf = dolfin.MeshFunction("size_t", mesh, mesh.topology().dim()-1) # MG20180418: size_t looks like unisgned int, but more robust wrt architecture and os
    boundaries_mf.set_all(0)

    xmin_sd.mark(boundaries_mf, xmin_id)
    xmax_sd.mark(boundaries_mf, xmax_id)
    ymin_sd.mark(boundaries_mf, ymin_id)
    ymax_sd.mark(boundaries_mf, ymax_id)

    ################################################################ Problem ###

    problem = dmech.MicroPoroHyperelasticityProblem(
        mesh=mesh,
        mesh_bbox=bbox,
        vertices=vertices,
        boundaries_mf=boundaries_mf,
        displacement_perturbation_degree=2,
        quadrature_degree=3,
        solid_behavior=mat_params,
        bcs=bcs)

    ################################################################ Loading ###

    Deltat = step_params.get("Deltat", 1.)
    dt_ini = step_params.get("dt_ini", 1.)
    dt_min = step_params.get("dt_min", 1.)
    dt_max = step_params.get("dt_max", 1.)
    k_step = problem.add_step(
        Deltat=Deltat,
        dt_ini=dt_ini,
        dt_min=dt_min,
        dt_max=dt_max)

    problem.add_surface_pressure_loading_operator(
        measure=problem.dS(0),
        P_ini=0., P_fin=pf,
        k_step=k_step)

    for k in range(dim):
        for l in range (dim):
            if (sigma_bar[k][l] is not None):
                problem.add_macroscopic_stress_component_constraint_operator(
                    i=k, j=l,
                    sigma_bar_ij_ini=0.0, sigma_bar_ij_fin=sigma_bar[k][l],
                    pf_ini=0.0, pf_fin=pf,
                    k_step=k_step)

    if (Macroscopic_strain is not None):
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=0, j=0,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_xx,
            pen_val=1e6,
            k_step=k_step)
        problem.add_macroscopic_stretch_component_penalty_operator(
            i=1, j=1,
            U_bar_ij_ini=0.0, U_bar_ij_fin=eps_yy,
            pen_val=1e6,
            k_step=k_step)

    ################################################################# Solver ###

    solver = dmech.NonlinearSolver(
        problem=problem,
        parameters={
            "sol_tol":[1e-6]*len(problem.subsols),
            "n_iter_max":32},
        relax_type="constant",
        write_iter=0)

    integrator = dmech.TimeIntegrator(
        problem=problem,
        solver=solver,
        parameters={
            "n_iter_for_accel":4,
            "n_iter_for_decel":16,
            "accel_coeff":2,
            "decel_coeff":2},
        print_out=res_basename*verbose,
        print_sta=res_basename*verbose,
        write_qois=res_basename+"-qois",
        write_qois_limited_precision=1,
        write_sol=res_basename*verbose)

    success = integrator.integrate()
    assert (success),\
        "Integration failed. Aborting."

    integrator.close()
    
    for operator in problem.operators: 
        if hasattr(operator, "material"):
            material = operator.material
            break
    
    U_bar = problem.get_macroscopic_stretch_subsol().func.vector().get_local().reshape((2,2))
    F_bar = U_bar + numpy.eye(2)
    J_bar = numpy.linalg.det(F_bar)
    C_bar = F_bar.T * F_bar
    v = J_bar * vol
    vs = dolfin.assemble(problem.kinematics.J * problem.dV)
    vf = v - vs
    
    Phi_s = vs/vol

    sigma_tot_xx = float((dolfin.assemble(material.sigma[0,0] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[0,0])/v)
    sigma_tot_yy = float((dolfin.assemble(material.sigma[1,1] * problem.kinematics.J * dV) - vf * pf *dolfin.Identity(2)[1,1])/v)

    return sigma_tot_xx

## Macro-micro model comparision 

In [ ]:
sigma_bar = [[None, 0.],
             [0., None]]

## $\sigma - E$

In [ ]:
sigma_micro = []
eps_micro = []

for i in range(21):
    eps = i/20
    eps_micro.append(eps)
    sigma_micro.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=eps, eps_yy=eps, pf=0., Macroscopic_strain=1, sigma_bar=sigma_bar, gamma=None)/Es)

In [ ]:
lam_lin = scipy.lambdify(epsilon, sigma_lin, modules=['numpy'])
lam = scipy.lambdify(epsilon, sigma_macro.subs(p, 0)[0,0], modules=['numpy'])

epsilon_vals = numpy.linspace(0, 1, 100)

sigma_macro_vals = lam(epsilon_vals)
sigma_vals_lin = lam_lin(epsilon_vals)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$E_x, E_y~()$', fontsize=16)
plt.ylabel(r'$\sigma/E_s~()$', fontsize=16)
plt.plot(epsilon_vals, sigma_vals_lin, '#005824')
plt.plot(eps_micro, sigma_micro, '#084594')
plt.plot(epsilon_vals, sigma_macro_vals, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])
plt.xlim(0, 0.5)
plt.ylim(0, 0.15)
plt.show()
# plt.savefig('macro-micro_small.pdf',bbox_inches='tight')

## $\sigma - p_f$

In [ ]:
p_lst = []
sigma_xx_lst = []
sigma_bar = [[0., 0.],
             [0., 0.]]

for i in range(11):
    p_ = i/10
    p_lst.append(p_)
    sigma_xx_lst.append(global_response(mesh=mesh, mat_params=mat_params, eps_xx=0.0, eps_yy=0.0, pf=p_, Macroscopic_strain=1, sigma_bar=sigma_bar, gamma=None)/Es)

In [ ]:
sigma_lin_p = -p

lam_in = scipy.lambdify(p, sigma_lin_p, modules=['numpy'])
lam = scipy.lambdify(p, sigma_macro[0,0].subs(epsilon, 0), modules=['numpy'])

p_vals = numpy.linspace(0, 1, 100)
sigma_xx_lam_lin = lam_in(p_vals)
sigma_xx_lam = lam(p_vals)

plt.figure()
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('legend', fontsize=12)
plt.xlabel(r'$p_f~(kPa)$', fontsize=16)
plt.ylabel(r'$\sigma/{E_s}~()$', fontsize=16)
plt.plot(p_vals, sigma_xx_lam_lin, '#005824')
plt.plot(p_lst, sigma_xx_lst, '#084594')
plt.plot(p_vals, sigma_xx_lam, '#99000D')
plt.legend(['Linear model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Microscopic model '+ r'$\tilde{\sigma}_{xx},~\tilde{\sigma}_{yy}$', 'Macroscopic model '+ r'$\bar{\sigma}_{xx},~\bar{\sigma}_{yy}$'])

plt.xlim(0, 0.5)
plt.ylim(-0.5, 0)
plt.show()
# plt.savefig('pf_sigma_incompressible.pdf', bbox_inches='tight')